<a href="https://colab.research.google.com/github/tchih11/qiita_eda/blob/main/notebooks/03_eda_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://{user_name}:@github.com/tchih11/qiita_eda.git

Cloning into 'qiita_eda'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 6), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [2]:
import os
!git clone https://github.com/Bjarten/early-stopping-pytorch.git
os.rename('early-stopping-pytorch','early_stopping_pytorch')

Cloning into 'early-stopping-pytorch'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Unpacking objects: 100% (92/92), done.


In [3]:
# 乱数シードの固定
import os
import random
import numpy as np
import torch
import tarfile
import os
import glob
import pandas as pd

SEED_VALUE = 0  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [4]:
%%capture
!pip install transformers==3.5.1
!pip install fugashi
!pip install ipadic

In [5]:
import glob
import gzip
import os
import random
import shutil
import sqlite3
import tarfile

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torchtext

from early_stopping_pytorch.pytorchtools import EarlyStopping
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
from tqdm import tqdm
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

from torch.utils.data import TensorDataset, Subset
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification

In [ ]:
pretrained_model = 'cl-tohoku/bert-base-japanese-char-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model)
model = BertModel.from_pretrained(pretrained_model)
batch_size = 16
max_length = 512

In [ ]:
def tokenizer_512(input_text):
    return tokenizer.encode_plus(
                    input_text,                      
                    add_special_tokens = True, # Special Tokenの追加
                    max_length = max_length,           # 文章の長さを固定（Padding/Trancatinating）
                    padding = "max_length", # PADDINGで埋める
                    truncation=True,
                    return_tensors = 'pt',     #  Pytorch tensorsで返す
                )["input_ids"][0]

def make_torch_dataset(df):

    label = df.label_index
    input_ids = []
    for item in df.text.apply(tokenizer_512):
        text = item.view(1, -1)
        input_ids.append(text)
        
    ids = torch.cat(input_ids, dim=0)
    label = torch.tensor(list(label))
    
    return TensorDataset(ids, label)

def make_dataloader(train_eval_df, test_df):
    torch.backends.cudnn.deterministic = True
    random.seed(1)
    torch.manual_seed(1)
    train_eval_df.reset_index(drop=True,inplace=True)
    train, eval = train_test_split(train_eval_df.label_index,stratify=train_eval_df.label_index,random_state=0,test_size=0.2)
    train_df = train_eval_df.loc[train.index]
    eval_df = train_eval_df.loc[eval.index]

    dataset_train = make_torch_dataset(train_df)
    dataset_eval = make_torch_dataset(eval_df)
    dataset_test = make_torch_dataset(test_df)
    
    dl_train = DataLoader(dataset_train,batch_size = 16)
    dl_eval = DataLoader(dataset_eval,batch_size = 16)
    dl_test = DataLoader(dataset_test,batch_size = 16)

    return dl_train, dl_eval, dl_test

def train_model(dl_train, dl_eval, net, device, batch_size, criterion, optimizer, patience, n_epochs=20):

    net.to(device)
    batch_size = dl_train.batch_size

    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = [] 
    
    early_stopping = EarlyStopping(patience=patience, delta=0.005, verbose=True)
    for epoch in tqdm(range(1, n_epochs + 1),total=n_epochs,position=0 ,leave=True, desc="train"):
        net.train() # prep model for training
        for batch in dl_train:
            data = batch[0].to(device)  # 文章
            target = batch[1].to(device)  # ラベル
            optimizer.zero_grad()
            output = net(data)[0]
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        net.eval() # prep model for evaluation
        for batch in dl_eval:
            data = batch[0].to(device)  # 文章
            target = batch[1].to(device)  # ラベル
            output = net(data)[0]
            loss = criterion(output, target)
            valid_losses.append(loss.item())

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, net)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    net.load_state_dict(torch.load('checkpoint.pt'))

    return  net, avg_train_losses, avg_valid_losses

def valid_test(net_trained, device, dl_test):
    net_trained.eval()   # モデルを検証モードに
    net_trained.to(device)  # GPU

    epoch_corrects = 0

    for batch in tqdm(dl_test,position=0 ,leave=True,desc="predict"):  # testデータのDataLoader
        data = batch[0].to(device)  # 文章
        target = batch[1].to(device)  # ラベル

        with torch.set_grad_enabled(False):
            outputs = net_trained(data)[0]
            _, preds = torch.max(outputs, 1)  # ラベルを予測
            epoch_corrects += torch.sum(preds == target)  # 正解数の合計を更新

    epoch_acc = epoch_corrects.double() / len(dl_test.dataset)
    return epoch_acc

In [ ]:
def modeling(train_eval_df,test_df,num_epochs=20):
    
    # GPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # dataloader作成
    dl_train, dl_eval, dl_test = make_dataloader(train_eval_df,test_df)
    
    # モデル構築
    net = BertForSequenceClassification.from_pretrained(pretrained_model, num_labels=9)
    net.train()

    # 重みを変更する個所
    for param in net.parameters():
        param.requires_grad = False
    for param in net.bert.encoder.layer[-1].parameters():
        param.requires_grad = True
    for param in net.classifier.parameters():
        param.requires_grad = True

    # 最適化手法
    optimizer = optim.Adam([
        {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
        {'params': net.classifier.parameters(), 'lr': 1e-4}
    ])

    # 損失関数
    criterion = nn.CrossEntropyLoss()      
    
    # 訓練実施
    net_trained,_,_ =  train_model(dl_train, dl_eval, net, device, batch_size, criterion, optimizer, 3, num_epochs)
    
    # テストデータで検証
    epoch_acc = valid_test(net_trained, device, dl_test)
    
    return epoch_acc.item()

In [ ]:
# test
test_df = pd.read_pickle("/content/qiita_eda/data/test_df.pkl")
test_df.columns = ["text","label_index"]

In [ ]:
pickles_path = [
                "/content/qiita_eda/data/train_eval_eda_2000_5_16_gzip.pkl",
                # "/content/qiita_eda/data/train_eval_eda_2000_10_16_gzip.pkl"
                ]
# alpha_list = [5,10]
alpha_list = [5]
sampling_list = [500,1000,2000]
num_agg_list = [0,1,4,9,16]

In [ ]:
result = pd.DataFrame(columns=["sampling_n","alpha","num_agg","accuracy","size"])

# pickle読み込み
for path,alpha in zip(pickles_path,alpha_list):
    
    train_eval_all = pd.read_pickle(path,compression="gzip")
    train_eval_all.reset_index(drop=True,inplace=True)

    unique_text_ids = list(train_eval_all.text_id.unique())
    raw_aug_id = train_eval_all.loc[train_eval_all.raw_flg==1,"aug_id"].unique()[0]

    # 件数を絞る
    for sampling_n in sampling_list:
        
        random.seed(0)
        text_id_list = random.sample(unique_text_ids,sampling_n)
        text_id_sampled = train_eval_all[train_eval_all.text_id.isin(text_id_list)]
        
        for i ,num_agg in enumerate(num_agg_list):
            if alpha!=alpha_list[0] and num_agg==0:
                continue
            # n_augで絞る
            aug_ids = list(text_id_sampled.loc[text_id_sampled.raw_flg==0,"aug_id"].unique())
            random.seed(0)
            aug_id_list = random.sample(aug_ids,num_agg)
            aug_id_list.append(raw_aug_id)

            train_eval_df = text_id_sampled.loc[text_id_sampled.aug_id.isin(aug_id_list),["text","label_index"]]
            acc = modeling(train_eval_df,test_df)
            idx = len(result)
            result.loc[idx] = [sampling_n, alpha, num_agg, acc,len(train_eval_df)]
            display(result)